In [1]:
#|default_exp util_func

# Utilities

In [2]:
#|hide
from nbdev.showdoc import *

In [2]:
#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee
from datetime import datetime

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import const_vars as const

Read geodataframe

In [4]:
#|export

def read_gdf(geometry_path : str , #path to geometry file (.shp /.gpkg). Path string should be written with r, for example r"path/to/geom/geom.gpkg"
                  crs : str , #crs code , for example '4326'. the coordinate reference system code to be used
                  ):
  
  gdf = gpd.read_file(geometry_path)
  gdf = gdf.set_crs(crs)

  return gdf

In [5]:
gdf = read_gdf(geometry_path=r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg",
         crs='4326')

GeoDataFrame to FeatureCollection

In [6]:
#|export

def gdf_to_featureCollection(
  gdf : gpd.GeoDataFrame , # geopandas geodataframe data
  ):
  ''' convert geodataframe to ee FeatureCollection type, based on this post https://gis.stackexchange.com/questions/439924/convert-local-file-shp-csv-into-earth-engine-ee-object'''
  
  # convert to json
  geo_json = gdf.to_json()

  #convert to FeatureCollection
  featureCollection = ee.FeatureCollection(json.loads(geo_json))


  return featureCollection
  

In [8]:
fc = gdf_to_featureCollection(gdf)
fc.getInfo()

{'type': 'FeatureCollection',
 'columns': {'id': 'Integer', 'name': 'Integer', 'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[34.1987406954507, 31.596126146017752],
      [34.19186977768542, 31.205628986357716],
      [34.60526999656305, 31.19646776267068],
      [34.63046336170241, 31.61101313450919],
      [34.1987406954507, 31.596126146017752]]]},
   'id': '0',
   'properties': {'id': 1, 'name': 1}}]}

Dates functions

In [ ]:
#|export
def extract_dates_from_img_coll(img):
    date = ee.Date(img.get(const.system_time_start_str)).format(const.date_format_1)
    return ee.Feature(None, {'date': date})

In [ ]:
#|export
def find_closest_date(user_date, date_list):
    # Convert user-provided date string to datetime object
    user_datetime = datetime.strptime(user_date, '%Y-%m-%d')

    # Convert the list of date strings to datetime objects
    date_objects = [datetime.strptime(date, '%Y-%m-%d') for date in date_list]

    # Find the closest date in the list
    closest_date = min(date_objects, key=lambda date: abs(date - user_datetime))

    # Convert the closest date back to string format
    closest_date_str = closest_date.strftime('%Y-%m-%d')

    return closest_date_str